In [1]:
# H2O AutoML - All Features
import h2o
from h2o.automl import H2OAutoML
from sklearn.model_selection import train_test_split
import pandas as pd

In [2]:
# Start H2O
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.441-b07, mixed mode)
  Starting server from C:\Users\John\anaconda3\envs\mlops_env\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\John\AppData\Local\Temp\tmp1myrlq50
  JVM stdout: C:\Users\John\AppData\Local\Temp\tmp1myrlq50\h2o_John_started_from_python.out
  JVM stderr: C:\Users\John\AppData\Local\Temp\tmp1myrlq50\h2o_John_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,America/Chicago
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.7
H2O_cluster_version_age:,3 months and 27 days
H2O_cluster_name:,H2O_from_python_John_thqvo4
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.485 Gb
H2O_cluster_total_cores:,24
H2O_cluster_allowed_cores:,24
H2O_cluster_status:,"locked, healthy"


In [3]:
# Load dataset
df = pd.read_csv("processed_athletes.csv")
df.head(2)

,upd_gender,upd_howlong,BMI,candj_rel,snatch_rel,deadlift_rel,backsq_rel,age,weight,total_lift
0,0,4,28.350347,1.536458,1.171875,2.421875,2.083333,35,192,1385
1,0,3,24.933391,1.548780,1.140244,2.420732,2.420732,27,164,1235


In [4]:
# Split manually
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Convert to H2OFrame
train_h2o = h2o.H2OFrame(train_df)
test_h2o = h2o.H2OFrame(test_df)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [5]:
target = "total_lift"
features = [col for col in df.columns if col != target]

In [6]:
# Run AutoML
aml_all = H2OAutoML(
    max_runtime_secs=300, 
    exclude_algos=["StackedEnsemble"], 
    seed=1
)
aml_all.train(x=features, y=target, training_frame=train_h2o)

AutoML progress: |
18:40:07.943: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_grid_1_AutoML_1_20250724_184007_model_2


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    557                557                         130753                 4            4            4             5             16            13.9874

ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 85.72723618799185
RMSE: 9.258900376826173
MAE: 7.248386367643742
RMSLE: NaN
Mean Residual Deviance: 85.72723618799185

ModelMetricsRegression: gbm
** Reported on cross-validation data. **

MSE: 185.19093363865693
RMSE: 13.608487558823608
MAE: 8.46538353422849
RMSLE: NaN
Mean Residual Deviance: 185.19093363865693

Cross-Validation Metrics Summary: 
                        mean       sd           cv_1_valid    cv_2_valid    cv_3_valid    cv_4_valid    cv_5_valid
----------------------  ---------  -----------  ------------  ------------  ------------  ------------  ------------
aic                     nan        0            nan           nan           nan           nan           nan
loglikelihood           nan        0            nan           nan           nan           nan           nan
mae                     8.46539    0.501074     9.10106       8.34811       7.71941       8.53405       8.62435
mean_residual_deviance  185.193    44.0763      230.085       153.495       133.698       230.473       178.211
mse                     185.193    44.0763      230.085       153.495       133.698       230.473       178.211
r2                      0.997582   0.000619848  0.99701       0.998028      0.998292      0.996881      0.997699
residual_deviance       185.193    44.0763      230.085       153.495       133.698       230.473       178.211
rmse                    13.5303    1.62907      15.1686       12.3893       11.5628       15.1813       13.3496
rmsle                   0.0227274  0.00557582   0.0222356     0.0285329     nan           0.0174138     nan

Scoring History: 
     timestamp            duration    number_of_trees    training_rmse       training_mae        training_deviance
---  -------------------  ----------  -----------------  ------------------  ------------------  -------------------
     2025-07-24 18:43:19  49.569 sec  0.0                277.3024258447387   229.8873903205842   76896.63537937682
     2025-07-24 18:43:19  49.603 sec  5.0                183.30801111096116  148.34483978864736  33601.82693745626
     2025-07-24 18:43:19  49.632 sec  10.0               127.07310462724955  99.89055680516051   16147.573919607912
     2025-07-24 18:43:19  49.661 sec  15.0               89.72435403540076   67.50981589395408   8050.4597070699365
     2025-07-24 18:43:19  49.688 sec  20.0               66.84056629880548   47.97790237831364   4467.661303145011
     2025-07-24 18:43:19  49.712 sec  25.0               52.04204848351924   36.0826736295848    2708.3748103609673
     2025-07-24 18:43:19  49.738 sec  30.0               42.852535549648884  29.064364403236898  1836.3398030339215
     2025-07-24 18:43:19  49.765 sec  35.0               36.86026645792489   24.80173853051775   1358.6792433492224
     2025-07-24 18:43:19  49.792 sec  40.0               32.603747653644106  22.040081659820473  1063.0043610625034
     2025-07-24 18:43:19  49.816 sec  45.0               29.542104141006227  20.216111603770898  872.7359170780574
---  ---                  ---         ---                ---                 ---                 ---
     2025-07-24 18:43:22  52.910 sec  515.0              9.573174362169231   7.486442867069922   91.64566736849426
     2025-07-24 18:43:22  52.963 sec  520.0              9.54312792867846    7.461305064324936   91.07129066312285


In [7]:
# Leaderboard
lb_all = aml_all.leaderboard
print("Top Models (All Features):")
lb_all.head(rows=5)


Top Models (All Features):


model_id,rmse,mse,mae,rmsle,mean_residual_deviance
GBM_grid_1_AutoML_1_20250724_184007_model_2,13.6085,185.191,8.46538,nan,185.191
GBM_4_AutoML_1_20250724_184007,13.7961,190.332,7.99171,0.0290671,190.332
GBM_5_AutoML_1_20250724_184007,14.1044,198.935,8.22738,nan,198.935
GBM_3_AutoML_1_20250724_184007,14.4595,209.077,8.54891,nan,209.077
GBM_2_AutoML_1_20250724_184007,14.521,210.859,8.93181,nan,210.859


In [13]:
# Feature importance from top model
best_model = aml_all.leader
feat_importance = best_model.varimp(use_pandas=True)

if feat_importance is not None:
	top_5_feats = feat_importance["variable"].tolist()[:5]
	print("Top 5 Features:", top_5_feats)
else:
	top_5_feats = []
	print("Feature importance is not available for this model.")

Top 5 Features: ['upd_gender', 'weight', 'backsq_rel', 'deadlift_rel', 'candj_rel']


In [ ]:
# Run AutoML with top 5 features
aml_5 = H2OAutoML(
    max_runtime_secs=300, 
    exclude_algos=["StackedEnsemble"], 
    seed=1
)
aml_5.train(x=top_5_feats, y=target, training_frame=train_h2o)

AutoML progress: |


19:03:43.282: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_5_AutoML_3_20250724_190343


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    208                208                         159485                 6            6            6             28            64            56.3365

ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 220.2369389127858
RMSE: 14.840382033923042
MAE: 11.416376636509682
RMSLE: 0.019359774109454157
Mean Residual Deviance: 220.2369389127858

ModelMetricsRegression: gbm
** Reported on cross-validation data. **

MSE: 381.90248941923863
RMSE: 19.54232558881462
MAE: 13.519427169195005
RMSLE: NaN
Mean Residual Deviance: 381.90248941923863

Cross-Validation Metrics Summary: 
                        mean       sd           cv_1_valid    cv_2_valid    cv_3_valid    cv_4_valid    cv_5_valid
----------------------  ---------  -----------  ------------  ------------  ------------  ------------  ------------
aic                     nan        0            nan           nan           nan           nan           nan
loglikelihood           nan        0            nan           nan           nan           nan           nan
mae                     13.5194    0.224358     13.7703       13.3674       13.2154       13.6522       13.5919
mean_residual_deviance  381.904    47.1494      428.286       344.688       334.739       435.613       366.195
mse                     381.904    47.1494      428.286       344.688       334.739       435.613       366.195
r2                      0.995022   0.000715425  0.994434      0.995572      0.995725      0.994104      0.995272
residual_deviance       381.904    47.1494      428.286       344.688       334.739       435.613       366.195
rmse                    19.5129    1.20031      20.6951       18.5658       18.2959       20.8713       19.1362
rmsle                   0.0298244  0.00716067   nan           0.0364156     0.0308522     0.0222053     nan

Scoring History: 
     timestamp            duration    number_of_trees    training_rmse       training_mae        training_deviance
---  -------------------  ----------  -----------------  ------------------  ------------------  -------------------
     2025-07-24 19:04:16  4.725 sec   0.0                277.3024258447387   229.8873903205842   76896.63537937682
     2025-07-24 19:04:16  4.783 sec   5.0                172.94434381950697  141.7514759872456   29909.746059159843
     2025-07-24 19:04:16  4.828 sec   10.0               109.48411430280365  88.3727126920848    11986.771284669374
     2025-07-24 19:04:16  4.896 sec   15.0               74.19548509182606   58.468835763757     5504.970008011383
     2025-07-24 19:04:16  4.935 sec   20.0               50.26812153433455   38.62698935025321   2526.8840425906287
     2025-07-24 19:04:16  4.983 sec   25.0               36.07464659041933   27.06355759071598   1301.3801266236528
     2025-07-24 19:04:16  5.020 sec   30.0               28.35320856111887   21.028550053091415  803.9044357103043
     2025-07-24 19:04:16  5.062 sec   35.0               24.722958554983197  18.283817218055944  611.2246797114168
     2025-07-24 19:04:16  5.111 sec   40.0               22.214021407725447  16.494893595791527  493.4627471028844
     2025-07-24 19:04:16  5.161 sec   45.0               20.59091482135036   15.415585812286528  423.98577318010587
---  ---                  ---         ---                ---                 ---                 ---
     2025-07-24 19:04:17  6.008 sec   165.0              15.314426647017152  11.77418097841751   234.531663526869
     2025-07-24 19:04:17  6.055 sec   170.0              15.275446216488808  11.740378145483447  233.339257112

In [ ]:
# Leaderboard
aml_5 = aml_all.leaderboard
print("Top Models (Top 5 Features):")
aml_5.head(rows=5)

Top Models (All Features):


model_id,rmse,mse,mae,rmsle,mean_residual_deviance
GBM_grid_1_AutoML_1_20250724_184007_model_2,13.6085,185.191,8.46538,nan,185.191
GBM_4_AutoML_1_20250724_184007,13.7961,190.332,7.99171,0.0290671,190.332
GBM_5_AutoML_1_20250724_184007,14.1044,198.935,8.22738,nan,198.935
GBM_3_AutoML_1_20250724_184007,14.4595,209.077,8.54891,nan,209.077
GBM_2_AutoML_1_20250724_184007,14.521,210.859,8.93181,nan,210.859
